In [1]:
import concurrent.futures
import jinja2
from anthropic import AnthropicBedrock
from compiler.core import Compiler
from tracing_client import TracingClient
from policies import common, handlers, typespec, drizzle, typescript, router, app_testcases

In [2]:
jinja_env = jinja2.Environment()
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = TracingClient(AnthropicBedrock(aws_profile="dev", aws_region="us-west-2"))

In [3]:
application_description = "Bot that greets user by name and returns personalized greeting."

content = jinja_env.from_string(typespec.PROMPT).render(application_description=application_description)
message = {"role": "user", "content": content}
with typespec.TypespecTaskNode.platform(client, compiler, jinja_env):
    tsp_data = typespec.TypespecTaskNode.run([message])
    tsp_root = typespec.TypespecTaskNode(tsp_data)
    tsp_solution = common.bfs(tsp_root)

match tsp_solution.is_successful:
    case True:
        print("Typespec solution is successful.")
    case False:
        raise Exception("Typespec solution is not successful.")

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


Typespec solution is successful.


In [4]:
print(tsp_solution.data.output.typespec_definitions)

model GreetingRequest {
    name: string;
    greeting: string;
}

interface GreetingBot {
    @llm_func(3)
    greet(options: GreetingRequest): string;
}


In [5]:
content = jinja_env.from_string(typescript.PROMPT).render(typespec_definitions=tsp_solution.data.output.typespec_definitions)
message = {"role": "user", "content": content}
with typescript.TypescriptTaskNode.platform(client, compiler, jinja_env):
    ts_data = typescript.TypescriptTaskNode.run([message])
    ts_root = typescript.TypescriptTaskNode(ts_data)
    ts_solution = common.bfs(ts_root)

match ts_solution.is_successful:
    case True:
        print("Typescript solution is successful.")
    case False:
        raise Exception("Typescript solution is not successful.")

Typescript solution is successful.


In [6]:
ts_solution.data.output.__dict__

{'reasoning': 'The application handles greeting requests with:\n1. A GreetingRequest model containing name and greeting fields\n2. A GreetingBot interface with a greet function that takes GreetingRequest and returns a string\n3. The function is marked as an LLM function with temperature 3',
 'typescript_schema': "import { z } from 'zod';\n\nexport const greetingRequestSchema = z.object({\n    name: z.string(),\n    greeting: z.string(),\n});\n\nexport type GreetingRequest = z.infer<typeof greetingRequestSchema>;\n\nexport declare function greet(options: GreetingRequest): Promise<string>;",
 'functions': [FunctionDeclaration(name='greet', argument_type='GreetingRequest', argument_schema='greetingRequestSchema', return_type='Promise<string>')],
 'type_to_zod': {'GreetingRequest': 'greetingRequestSchema'},
 'feedback': {'exit_code': 0, 'stdout': None, 'stderr': None}}

In [7]:
print(ts_solution.data.output.typescript_schema)

import { z } from 'zod';

export const greetingRequestSchema = z.object({
    name: z.string(),
    greeting: z.string(),
});

export type GreetingRequest = z.infer<typeof greetingRequestSchema>;

export declare function greet(options: GreetingRequest): Promise<string>;


In [8]:
content = jinja_env.from_string(drizzle.PROMPT).render(typespec_definitions=tsp_solution.data.output.typespec_definitions)
message = {"role": "user", "content": content}
with drizzle.DrizzleTaskNode.platform(client, compiler, jinja_env):
    dzl_data = drizzle.DrizzleTaskNode.run([message])
    dzl_root = drizzle.DrizzleTaskNode(dzl_data)
    dzl_solution = common.bfs(dzl_root)

match dzl_solution.is_successful:
    case True:
        print("Drizzle solution is successful.")
    case False:
        raise Exception("Drizzle solution is not successful.")

Drizzle solution is successful.


In [9]:
print(dzl_solution.data.output.drizzle_schema)

import { serial, text, pgTable, timestamp } from "drizzle-orm/pg-core";

export const greetingRequestsTable = pgTable("greeting_requests", {
  id: serial("id").primaryKey(),
  name: text("name").notNull(),
  greeting: text("greeting").notNull(),
  created_at: timestamp("created_at").defaultNow().notNull()
});

export const greetingResponsesTable = pgTable("greeting_responses", {
  id: serial("id").primaryKey(),
  request_id: serial("request_id")
    .references(() => greetingRequestsTable.id)
    .notNull(),
  response_text: text("response_text").notNull(),
  created_at: timestamp("created_at").defaultNow().notNull()
});


In [ ]:
def search_handler(
    function_name: str,
    argument_type: str,
    argument_schema: str,
    typespec_definitions: str,
    typescript_schema: str,
    drizzle_schema: str,
) -> handlers.HandlerTaskNode:
    prompt_params = {
        "function_name": function_name,
        "argument_type": argument_type,
        "argument_schema": argument_schema,
        "typespec_schema": typespec_definitions,
        "typescript_schema": typescript_schema,
        "drizzle_schema": drizzle_schema,
    }
    content = jinja_env.from_string(handlers.PROMPT).render(**prompt_params)
    message = {"role": "user", "content": content}
    output = handlers.HandlerTaskNode.run([message], **prompt_params)
    root_node = handlers.HandlerTaskNode(output)
    return root_node # No BFS for now
    #solution = common.bfs(root_node)
    #return solution

In [11]:
handler_target = ts_solution.data.output.functions[0]

In [12]:
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    handler_solution = search_handler(
        function_name=handler_target.name,
        argument_type=handler_target.argument_type,
        argument_schema=handler_target.argument_schema,
        typespec_definitions=tsp_solution.data.output.typespec_definitions,
        typescript_schema=ts_solution.data.output.typescript_schema,
        drizzle_schema=dzl_solution.data.output.drizzle_schema,
    )

In [13]:
handler_solution.data.output.__dict__

{'handler': 'import { db } from "../db";\nimport type { GreetingRequest } from "../common/schema";\nimport { greetingRequestsTable, greetingResponsesTable } from "../db/schema/application";\n\nexport const handle = async (options: GreetingRequest): Promise<string> => {\n    // Insert the greeting request\n    const [insertedRequest] = await db\n        .insert(greetingRequestsTable)\n        .values({\n            name: options.name,\n            greeting: options.greeting,\n        })\n        .returning();\n\n    // Construct the response text\n    const responseText = `${options.greeting}, ${options.name}!`;\n\n    // Store the response\n    await db\n        .insert(greetingResponsesTable)\n        .values({\n            request_id: insertedRequest.id,\n            response_text: responseText,\n        });\n\n    return responseText;\n};',
 'feedback': {'exit_code': 0, 'stdout': None, 'stderr': None}}

In [14]:
print(handler_solution.data.output.feedback["stdout"])

None


In [15]:
print(handler_solution.data.output.handler)

import { db } from "../db";
import type { GreetingRequest } from "../common/schema";
import { greetingRequestsTable, greetingResponsesTable } from "../db/schema/application";

export const handle = async (options: GreetingRequest): Promise<string> => {
    // Insert the greeting request
    const [insertedRequest] = await db
        .insert(greetingRequestsTable)
        .values({
            name: options.name,
            greeting: options.greeting,
        })
        .returning();

    // Construct the response text
    const responseText = `${options.greeting}, ${options.name}!`;

    // Store the response
    await db
        .insert(greetingResponsesTable)
        .values({
            request_id: insertedRequest.id,
            response_text: responseText,
        });

    return responseText;
};
